## Example script | PyFrag Plotter
This notebook acts as an example of how to use the "pyfrag_plotter" package to plot the results of a PyFrag calculation.

### Prequirements:
- Installation of the "pyfrag_plotter" package (e.g. `python -m pip install pyfrag_plotter`)

### Requirements for running the script: 
- Directory containing the relevant PyFrag files, being the inputfile (.in) and the outputfile (pyfrag_*.txt)

### Flow of the program:
- Initialize the config file
- Load the PyFrag output file and input file. This will create a dataframe that contains all the information from the PyFrag output file, and a dictionary that contains all the information from the PyFrag input file.
- Process the dataframe by trimming data, removing dispersion key, removing outliers and more
- Make a PyFragResults object that combines the specifications in the PyFrag input file with the unclear header keys (such as overlap_1) in processed dataframe

From here, you can choose to plot or to interpolate. The plotting function will plot the results of the PyFrag calculation, and the interpolation function will interpolate at a specified point of on the chosen IRC coordinate. The interpolation function is not yet implemented.

## Start the program!

First, Initialize the config file. In this file, you can specify plot settings, such as the font, font size, figure size, etc, but also which eda keys to plot, and until which energy value (dEint) to plot.

If you have a specific config file you want to use, you can provide it as an argument to the initialize_pyfrag_plotter function. If you don't provide a config file, the default config file will be used that is part of the package.

In [1]:
# Load the modules
import os
from pyfrag_plotter.config_handler import initialize_pyfrag_plotter
from pyfrag_plotter.file_func import get_pyfrag_files
from pyfrag_plotter.input.read_inputfile import read_inputfile
from pyfrag_plotter.input.read_resultsfile import read_data
from pyfrag_plotter.processing_funcs import trim_data, remove_dispersion_term
from pyfrag_plotter.pyfrag_object import create_pyfrag_object
from pyfrag_plotter.plot.plotter import Plotter

Now it is time to load the PyFrag output file and input file. This will create a dataframe that contains all the information from the PyFrag output file, and a dictionary that contains all the information from the PyFrag input file.

In [2]:
# First, initialize the config file
initialize_pyfrag_plotter()

# Next, specify the path to the pyfrag output directory and read in the input file and the output file
base_path = "/Users/siebeld/Library/CloudStorage/OneDrive-VrijeUniversiteitAmsterdam/PhD/Scripting/local_packages/theochem/example"
result_dirs = ["ureas_di_O_Cs_all", "ureas_di_O_Cs_sigma", "ureas_di_O_Cs_pi"]
pyfrag_dirs = [os.path.join(base_path, dir) for dir in result_dirs]

# Get the pyfrag input file and pyfrag output file
pyfrag_files = get_pyfrag_files(pyfrag_dirs)
input_files, results_files = zip(*pyfrag_files)

# Read the input file
input_contents = [read_inputfile(input_file) for input_file in input_files]

# Read the output file
output_contents = [read_data(results_file) for results_file in results_files]


Since the input file(s) and output file(s) are loaded, we can process them and make a PyFragResults object. This object combines the specifications in the PyFrag input file with the unclear header keys (such as overlap_1) in processed dataframe.

In [3]:
# Now, we can start processing the data. For example, we can trim the data up to the minimum energy value. Also, we can remove outliers or remove the dispersion keyword from the data.
output_contents = [trim_data(output_content) for output_content in output_contents]
output_contents = [remove_dispersion_term(output_content) for output_content in output_contents]
# print(output_content["EnergyTotal"])

objs = [create_pyfrag_object(output_content, input_content) for output_content, input_content in zip(output_contents, input_contents)]


Now let's do some plotting! Make an instance of the MultiPlotter by providing the directory in which the data is stored and the name of the corresponding output folder. Also, provide the PyFragResultsObjects and the irc coordinate on which the data should be plotted. The Multiplotter is able to plot ASM, EDA and ASM extra strain curves. You can even provide the keys that should be plotted, such as "EnergyTotal" or "Int" (they should match the keys in the pyfrag_*.txt resultsfile).

In [4]:
plot_dir = "/Users/siebeld/Desktop"
plot_inst = Plotter("plot_all_O_tri_ureas", plot_dir, objs, ("bondlength",  "r - r$_{eq}$ / Å"))
plot_inst.plot_asm()  # ["EnergyTotal"]
plot_inst.plot_eda()
plot_inst.plot_extra_strain()

<Figure size 1200x1000 with 0 Axes>

You can also plot overlaps, populations and orbital energies in combinations you desire! Just pass the desired quantities as a list to the plot function. For example, if you want to plot the overlap and the populations, you can do it as follows: